## Homework 3
<br>Kennon Stewart
ksstewar@umich.edu

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [2]:
college_test_path = "/workspaces/stats_503/homework_3/college_test.csv"
college_train_path = "/workspaces/stats_503/homework_3/college_train.csv"

college_train = pd.read_csv(college_test_path)
college_test = pd.read_csv(college_train_path)

In [3]:
X_college_train = college_train.drop(['Name', 'Private'], axis=1)
y_college_train = college_train['Private']

X_college_test = college_test.drop(['Name', 'Private'], axis=1)
y_college_test = college_test['Private']

In [4]:
# initialize the model
model = LogisticRegression(random_state = 9271999)

# fit to the parameters naively
model.fit(X_college_train, y_college_train)

# let it do

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=9271999)

In [5]:
y_pred = model.predict(X_college_test)
y_pred


array(['Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes',
       'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes',
       'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes',
       'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No',
       'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes',
       'Yes', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes',
       'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No',
       'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No',
       'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes',
       'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes',
       'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes',
       'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes',
       'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No',
       'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', '

In [6]:
prediction_probabilities = model.predict_proba(X_college_test)

sum = 0
for no_prob, yes_prob in prediction_probabilities:
    sum -= np.log(max(yes_prob, no_prob))
print(sum)

2.902610293810998


In [7]:
def hard_classification(threshold: int, probabilities:np.ndarray) -> np.array:
    results = [0] * len(probabilities)
    for ind in range(len(probabilities)):
        if probabilities[ind][1] >= threshold:
            results[ind] = 1
        else:
            results[ind] = 0
    return results

point_5_classifications = hard_classification(0.5, prediction_probabilities)
point_9_classifications = hard_classification(0.9, prediction_probabilities)

In [8]:
def confusion_matrix(y_true:np.array, y_predicted:np.array) -> np.ndarray:
    FP = TP = FN = TN = 0
    y_true = [1 if prediction == "Yes" else 0 for prediction in y_true]
    for prediction, truth in zip(y_predicted, y_true):
        if (prediction == 1) and (truth == 1): # true positive
            TP += 1
        elif (prediction == 1) and (truth == 0): # false positive
            FP += 1
        elif (prediction == 0) and (truth == 0): # true negative
            TN += 1
        else: # false negative
            FN += 1
    conf_matrix = np.array([[TN, FP], [FN, TP]])

    return conf_matrix

point_5_results = confusion_matrix(y_college_test, point_5_classifications)
point_9_results = confusion_matrix(y_college_test, point_9_classifications)
print(point_5_results)
print(point_9_results)

[[139  40]
 [ 36 435]]
[[143  36]
 [ 38 433]]


In [9]:
def rate_calculations(test_set, classifications) -> tuple:
    # calculate the confusion matrix
    matrix = confusion_matrix(test_set, classifications)

    # assign the TP, FP, TN, FN rates
    TP = int(matrix[0][0])
    FP = int(matrix[0][1])
    TN = int(matrix[1][0])
    FN = int(matrix[1][1])

    # status check
    print(f"TP: {TP}")
    print(f"FP: {FP}")
    print(f"TN: {TN}")
    print(f"FN: {FN}")

    # calculate TPR
    TPR = TP / max((TP + FN), 1)
    # calculate FPR
    FPR = FP / max((FP + TN), 1)

    # calculate TNR
    TNR = TN / max((TN + FP), 1)

    # calculate FNR
    FNR = FN / max((FN + TN), 1)

    return (TPR, FPR, TNR, FNR)

point_5_results = rate_calculations(y_college_test, point_5_classifications)

TP: 139
FP: 40
TN: 36
FN: 435


In [10]:
point_5_results

(0.2421602787456446,
 0.5263157894736842,
 0.47368421052631576,
 0.9235668789808917)

In [11]:
def simulate_thresholds():   
    # initialize the range of thresholds
    thresholds = np.linspace(0, 1, 101)
    results = list()
    for threshold in thresholds:
        classifications = hard_classification(threshold, prediction_probabilities)# compute the classifications

        confusion_matrix_object = confusion_matrix(y_college_test, classifications) # compute the confusion matrix

        # store the results
        FP, TP = confusion_matrix_object[0][1], confusion_matrix_object[1][1]
        results.append((int(FP), int(TP)))

    # return the results
    return results

simulate_thresholds()

[(179, 471),
 (48, 438),
 (47, 437),
 (45, 437),
 (44, 437),
 (43, 437),
 (43, 437),
 (43, 437),
 (43, 436),
 (43, 436),
 (43, 436),
 (41, 436),
 (41, 436),
 (41, 436),
 (40, 436),
 (40, 436),
 (40, 436),
 (40, 436),
 (40, 436),
 (40, 436),
 (40, 436),
 (40, 436),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (40, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (38, 435),
 (37, 435),
 (37, 435),
 (37, 435),
 (37, 435),
 (37, 435),
 (3